# Import

In [1]:
from src.solver.solver_ortools import solve as ortools_solve
from src.solver.solver_tsptw import solve
from src.problem.tsptw.environment.tsptw import TSPTW
from src.real_world import (
    generate_field_instance,
    to_real_coord,
    to_virtual_coord,
    calc_lat_lon,
    REAL_X_RANGE,
    REAL_Y_RANGE,
    to_real_time,
    to_virtual_time
)
import folium
from folium.features import DivIcon
import numpy as np


# TSPTW問題の生成(in 文京区)

In [2]:
# Constant
radius = 10
n_city = 20
grid_size = 100
solvable = False
time_window_size = 2 * 60  # 2時間
instance = generate_field_instance(n_city, grid_size, solvable=solvable, max_tw_size=to_virtual_time(time_window_size))

### 描画関数

In [3]:
def display_map(instance: TSPTW, solution: list):
    center = calc_lat_lon(np.mean(REAL_X_RANGE), np.mean(REAL_Y_RANGE))
    m = folium.Map(location=center, zoom_start=15)

    locations = list()
    for city in solution:
        x = instance.x_coord[city]
        y = instance.y_coord[city]
        window = instance.time_windows[city]
        r_x, r_y = to_real_coord(x, y)
        location = calc_lat_lon(r_x, r_y)
        locations.append(location)
        folium.CircleMarker(location=location, radius=radius,
                            color='blue').add_to(m)
        window_start = int(round(to_real_time(window[0])))
        window_end = int(round(to_real_time(window[1])))
        text = "T:[" + str(window_start) + ", " + str(window_end) + "]"
        folium.map.Marker(
            location,
            icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(0,0),
                html='<div style="font-size: 10pt; color: RED">%s</div>' % text,
                )
            ).add_to(m)
    folium.PolyLine(locations=locations).add_to(m)
    display(m)

### 解の評価指標生成関数

In [4]:
def solution_indicator(instance: TSPTW, solution: list):
    # 評価指標
    wait = 0
    delay = 0
    cur_time = 0
        
    pre_x, pre_y = instance.x_coord[solution[0]], instance.y_coord[solution[0]]
    for city in solution:
        x = instance.x_coord[city]
        y = instance.y_coord[city]
        window = instance.time_windows[city]  # [t_start, t_end]
        distance = ((x - pre_x) ** 2 + (y - pre_y) ** 2) ** 0.5
        tmp_time = cur_time + distance
        if tmp_time < window[0]:
            wait += window[0] - tmp_time
            cur_time = window[0]
        elif tmp_time > window[1]:
            delay += tmp_time - window[1]
            cur_time = tmp_time
        else:
            cur_time = tmp_time
        pre_x, pre_y = x, y
    return cur_time, wait, delay

# 強化学習で求解

In [5]:
path = f"instances/tsptw/{n_city}.json"
instance.save(path)
solution = solve(n_city, path)
solution = [0] + solution + [0]
if len(solution) > 2:
    time, wait, delay = solution_indicator(instance, solution)
    time, wait, delay = map(to_real_time, [time, wait, delay])
    time, wait, delay = map(round, [time, wait, delay])
    time, wait, delay = map(int, [time, wait, delay])

    print(f"配送時間：{time}分　待機時間：{wait}分　遅延時間：{delay}分")
    display_map(instance, solution)
else:
    print("実行可能な配送経路を見つけられません")

配送時間：376分　待機時間：201分　遅延時間：0分


# Google OR-Toolsで求解

In [6]:
or_solution = ortools_solve(instance)
if len(or_solution) > 2:
    time, wait, delay = solution_indicator(instance, or_solution)
    time, wait, delay = map(to_real_time, [time, wait, delay])
    time, wait, delay = map(round, [time, wait, delay])
    time, wait, delay = map(int, [time, wait, delay])

    print(f"配送時間：{time}分　待機時間：{wait}分　遅延時間：{delay}分")
    display_map(instance, or_solution)
else:
    print("実行可能な配送経路を見つけられません")

実行可能な配送経路を見つけられません


## TODO
- 時間制約を色で可視化(丸をグラデーションlike 白 to 黒)
- 端から端への考察（はしはしが何個あって、それぞれの理由を考察、近くに時間制約上都合の良いノードがないことを確認＆可視化）
- 3D可視化が空間的な一筆書きっぽいものであることの確認
- 工夫の余地はどこから生まれるのかを考察
- 時間指定割合のコントロール(3-5割で分析)
- 時間幅のコントロール（30m, 2h, 4h)で考察
- Story作り


## 今後の課題として
- 軒先時間判定
- ノード数の一般化検証

## その場の話
- アカデミックな主張の仕方の検討
- その他建設的なアドバイスをもらうために利用する！